<a href="https://colab.research.google.com/github/ruadhan/datadag/blob/master/dogs_vs_cats1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Let op! Klik bovenin bij "Runtime" op "Change runtime type" en zet de "Hardware accelerator" op "GPU". Anders kan je een hoop kopjes koffie halen tijdens deze workshop.**

De data die we gebruiken komt van de Kaggle [Dogs vs Cats dataset](https://www.kaggle.com/c/dogs-vs-cats/data). Deze dataset bestaat uit 25.000 plaatjes van katten en honden. Omdat dit nogal veel is heb ik een kleine sample van 4000 plaatjes genomen en alvast in een nette mapstructuur gezet. Met onderstaand commando kan je ze snel downloaden naar je Colab omgeving.

In [0]:
!git clone https://github.com/ruadhan/dogs_vs_cats

We zetten even alle directories in variabelen zodat we er later makkelijk mee kunnen werken

In [0]:
train_dir = 'dogs_vs_cats/train'
validation_dir = 'dogs_vs_cats/validation'
test_dir = 'dogs_vs_cats/test'

train_cats_dir = 'dogs_vs_cats/train/cats'
train_dogs_dir = 'dogs_vs_cats/train/dogs'

validation_cats_dir = 'dogs_vs_cats/validation/cats'
validation_dogs_dir = 'dogs_vs_cats/validation/dogs'

test_cats_dir = 'dogs_vs_cats/test/cats'
test_dogs_dir = 'dogs_vs_cats/test/dogs'

Even checken hoeveel plaatjes in welke folder zitten

In [0]:
import os

print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

We gebruiken hier een de [ImageDataGenerator](https://keras.io/preprocessing/image/) van Keras om onze plaatjes makkelijk in het neurale netwerk te laden. (Later kunnen we deze ook gebruiken om extra plaatjes te genereren.)

In [0]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Nu komt het echte werk, jullie mogen zelf een convolutional neural network bouwen. Hiervoor gebruiken we een [sequential model](https://keras.io/getting-started/sequential-model-guide/), wat inhoud dat je laagje voor laagje je layers moet toevoegen. Voor extra hints is hier de documentatie van 
[convolutional layers](https://keras.io/layers/convolutional/), [max pooling layers](https://keras.io/layers/pooling/) en hier vind je de 
[flatten layer](https://keras.io/layers/core/). (Let op: de eerste laag is afwijkend en moet een input_shape variable bevatten)

In [0]:
from keras import layers, models

model = models.Sequential()
#TODO: Maak je model met de verschillende layers

Als je model af is kan je met deze code een samenvatting bekijken. Bij "Output Shape" kan je zien hoe groot je overgeleven plaatje nog is. Let op dat dit niet onder de nul komt.

In [0]:
model.summary()

We compileren het model met een bepaalde optimizer, en een loss function die bij ons probleem hoort.

In [0]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

Nu gaan we ons model 'fitten' aan de dataset. Let op, nu is het moment om koffie voor jezelf te zetten! Eventueel kan je spelen met de hyperparameters.

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

Omdat we dit niet weer willen meemaken slaan we het gefitte model op. (Let op, als je iets veranderd aan het netwerk moet je hem wel opnieuw fitten)

In [0]:
model.save('cats_and_dogs_small_1.h5')

En even plotten hoe goed/slecht ons model is.

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()